In [1]:
import weaviate
from llama_index.core import Settings, StorageContext, VectorStoreIndex
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.vector_stores.weaviate import WeaviateVectorStore

from src.indexing_utils import load_docs

Settings.embed_model = GeminiEmbedding(model_name='models/embedding-001', task_type="retrieval_query")
Settings.llm = Gemini(model_name='models/gemini-pro', temperature=0.5)

# Connect to local instance
client = weaviate.Client("http://localhost:8080")

/home/dai/anaconda3/envs/ragenv/lib/python3.11/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
/home/dai/anaconda3/envs/ragenv/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


### Load documents

In [2]:
nodes = load_docs(path="/home/dai/33/project/rag/storage/docstore/Articles_with_embeddings_on_chunks",
                  return_docstore=False)

In [3]:
vector_store = WeaviateVectorStore(weaviate_client=client, index_name="LlamaIndex_articles")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

In [4]:
# Query Index with Hybrid Search as chat engine
chat_engine = index.as_chat_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, alpha=0.5, chat_mode="condense_plus_context"
)

In [5]:
streaming_response = chat_engine.stream_chat("What is the Anukta Dravya Vichaara?")
for token in streaming_response.response_gen:
    print(token, end="")

Anukta Dravya Vichaara is an important Ayurvedic subject that investigates the properties of food items not directly or specifically mentioned in Ayurvedic texts. It involves examining the potential role of these foods in human diets.

In [6]:
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(streaming_response,show_source=True)

Final Response: Anukta Dravya Vichaara is an important Ayurvedic
subject that investigates the properties of food items not directly or
specifically mentioned in Ayurvedic texts. It involves examining the
potential role of these foods in human diets.
______________________________________________________________________
Source Node 1/2
Node ID: 39f5a408-f55b-4ccf-9f0a-12afcbfbcb4d
Similarity: 0.983606558
Text: Anukta Dravya Vichaara Vd.Shreeprasad Bavdekar  Ph.D. (Ayurveda)
Introduction: Ayurveda, as the name suggests, is the Veda of life. It
involves the discussion of every aspect related to human life. As
mentioned in Charaka Samhitaa, Hitaahitam Sukham Duhkham Aayustasya
Hitaahitam..C.Su. 1/41 [That is named the Veda of life, wherein is
told the g...
______________________________________________________________________
Source Node 2/2
Node ID: f0f89eca-5fa3-4878-8d1f-69bfff23d335
Similarity: 0.984122984
Text: If it is done, Sita milk in the hot state would become Viruddha,
and as t

/home/dai/anaconda3/envs/ragenv/lib/python3.11/site-packages/llama_index/legacy/download/module.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/dai/anaconda3/envs/ragenv/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
